In [0]:
import numpy as np 
import pandas as pd 
import os
from tqdm import tqdm, trange
import pickle
import gc


from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Input, Dense, Embedding, Dropout, add, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, LSTM, CuDNNGRU
from keras.losses import binary_crossentropy
from keras.models import Model
from keras.optimizers import Adam

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

Установим модуль для работы с BERT

In [0]:
!pip install pytorch-pretrained-bert

In [0]:
from pytorch_pretrained_bert import BertTokenizer
from pytorch_pretrained_bert.modeling import BertModel

Прежде, чем приступать к работе с моделью -- скачаем данные (уже обработанные). 

In [0]:
!wget https://www.dropbox.com/s/3w5swcdhhix2vkt/train_bert-base-uncased_ids.csv.zip

--2019-10-22 18:12:53--  https://www.dropbox.com/s/3w5swcdhhix2vkt/train_bert-base-uncased_ids.csv.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/3w5swcdhhix2vkt/train_bert-base-uncased_ids.csv.zip [following]
--2019-10-22 18:12:53--  https://www.dropbox.com/s/raw/3w5swcdhhix2vkt/train_bert-base-uncased_ids.csv.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2252f1c31f615c92e1cdc66fe4.dl.dropboxusercontent.com/cd/0/inline/Aq6psIuqdt9pW32wHG_QZLpN705CLi_OfylC9vCbgq5skimMzYLhy0H86zEQXavk6X90nyRyT7OqGBYl9yw1HNCRmyzadyOwHycgEhdbWbSYVp91WpGnJdfmsai-uNSyFUg/file# [following]
--2019-10-22 18:12:53--  https://uc2252f1c31f615c92e1cdc66fe4.dl.dropboxusercontent.com/cd/0/inline/Aq6psIuqdt9pW32wHG_QZLpN705CLi_OfylC9vCbgq

Теперь загрузим эмбеддинги из модели BERT

In [0]:
def get_bert_embed_matrix():
    bert = BertModel.from_pretrained('bert-base-uncased') #bert-large-uncased
    bert_embeddings = list(bert.children())[0] #get all modules
    bert_word_embeddings = list(bert_embeddings.children())[0] #choose the one that stores embedding
    embeddings = bert_word_embeddings.weight.data.numpy() #pytorch magic
    return embeddings

In [0]:
list(bert.children()

<generator object Module.children at 0x7f05c36f7f68>

In [0]:
train = pd.read_csv('train_bert-base-uncased_ids.csv.zip')
train = train.sample(frac=0.1)

In [0]:
embedding_matrix = get_bert_embed_matrix()

Загрузим токены в матрицу:

In [0]:
x_train = np.zeros((train.shape[0], 250),dtype=np.int)

for i, ids in tqdm(enumerate(list(train['comment_text']))):
    input_ids = [int(i) for i in ids.split()[:250]]
    inp_len = len(input_ids)
    x_train[i,:inp_len] = np.array(input_ids)
    
y_aux_train = train[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']].values    

180487it [00:05, 35077.30it/s]


### Задание 0. Разбейте выборку на train/val.

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(x_train, y_aux_train, random_seed=42)

### Задание 1: Напишите свою модель

Что в него должно входить?



*  Загрузка эммедингов из берта в качестве весов 
*  CuDNNLSTM / CuDNNGRU
*  Несколько полносвязных слоев
*  Клиппинг градиентов в оптимайзере


Пример: 

Input -> Embedding -> LSTM/GRU -> Dense -> Sigomoid(num_aux_targets)

In [0]:
def build_model(embedding_matrix, num_aux_targets):
    input_layer = Input((250,), name = 'comment_text')
    embedding_layer = Embedding(*embedding_matrix.shape, input_length=250, 
                                weights=[embedding_matrix], 
                                trainable = False)(input_layer)
    x = Bidirectional(CuDNNGRU(128, return_sequences=True))(embedding_layer)
    x = Dropout(0.3)(x)
    x = Bidirectional(CuDNNGRU(128, return_sequences=False))(x) #true
    #MaxPooling, AveragePooling
    x = Dense(64, activation="relu")(x)
    output_layer = Dense(num_aux_targets, activation="sigmoid")(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='binary_crossentropy',
                      optimizer=Adam(clipvalue=1, clipnorm=1),
                      metrics=['accuracy'])
    return model

In [0]:
model = build_model(embedding_matrix, y_aux_train.shape[1])

### Задание 3: Напишите свой fit

Что в него должно входить?



*   Сохранение чекпойнтов
*   LRScheduler 
*   Клиппинг градиентов
*   Замер качества каждую эпоху
*   Что-то что захочется вам

Используйте Keras callbacks

In [0]:
checkpointer = ModelCheckpoint(filepath = 'model_zero7.{epoch:02d}-{val_loss:.6f}.hdf5',
                               verbose=1,
                               save_best_only=False, save_weights_only = True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)

history = model.fit(X_train, y_train, batch_size=256, epochs = 7, 
                    validation_data=(X_val, y_val), verbose = 1, 
                    callbacks = [checkpointer, reduce_lr])



Train on 135365 samples, validate on 45122 samples
Epoch 1/7
135365/135365 [==============================] - 253s 2ms/step - loss: 0.1434 - acc: 0.8545 - val_loss: 0.1236 - val_acc: 0.8539

Epoch 00001: saving model to model_zero7.01-0.123619.hdf5
Epoch 2/7
135365/135365 [==============================] - 250s 2ms/step - loss: 0.1171 - acc: 0.8550 - val_loss: 0.1144 - val_acc: 0.8537

Epoch 00002: saving model to model_zero7.02-0.114356.hdf5
Epoch 3/7
121856/135365 [==========================>...] - ETA: 22s - loss: 0.1112 - acc: 0.8556

In [0]:
?model.fit

### Задание 4:

Усредните предсказания модели с нескольких чекпоинтов и сравните качество

In [0]:
#your code here

### Задание 5. NMT (бонус)

Напишите простую seq2seq сеть для задач машинного перевода.

Что в ней должно быть?

* Encoder (LSTM сеть, которая возвращает последнее скрытое состояние)
* Decoder (LSTM сеть, которая предсказывает токены в новом языке)


Прежде, чем строить сеть предобработайте данные: 



*  Токенизируйте
*  Сделайте паддинг
*  Загрузите эмбеддинги для энкодера. (как на прошлом семинаре)

In [0]:
!wget http://lotus.kuee.kyoto-u.ac.jp/WAT/WAT2017/snmt/small-NMT.tar.bz2
!tar -tf small-NMT.tar.bz2

In [0]:
!tar xvjf small-NMT.tar.bz2